# Interoperability

Here, we briefly demonstrate how the single-cell objects may be converted between R and python to run CCC analyses in either language:

In [ ]:
library(Seurat, quietly = TRUE)
library(SeuratDisk, quietly = T)
library(zellkonverter, quietly = TRUE)
library(reticulate, quietly = TRUE)
library('Matrix', quietly = T)
anndata<-import('anndata')

data.path <- file.path("..", "..", "data")

## to Python

For use in Python, we can convert the SingleCellExperiment object generated in [Tutorial 1](./01-Preprocess-Expression.ipynb) to an Anndata object using zellkonverter. See the [companion supplementary R tutorial](../ccc_python/S1_Interoperability.ipynb) for loading these saved files.

In [ ]:
covid_data<-readRDS(file.path(data.path, 'covid_balf_norm.rds'))

data.path<-'/data/hratch/ccc_protocols/interim/'
zellkonverter::writeH5AD(covid_data, 
                         X_name = 'logcounts', 
                         paste0(data.path, 'processed_fromR_toPython.h5ad'))

In [ ]:
file.name<-paste0(data.path, 'interim/', 'processed_fromR_toPython_Seurat.h5Seurat')
suppressMessages({
    SaveH5Seurat(covid_data, filename = file.name)
    Convert(file.name, dest = "h5ad", overwrite = TRUE)
})

## from Python

Here, we can load the expression matrices that were generated in the companion python script using AnnData and saved as h5ad files into Seurat:

Here, we can load the data that was generated in the [python version of Tutorial l](../ccc_python/01-Preprocess-Expression.ipynb) using scanpy into a SCE object:

In [ ]:
data_path = file.path("..", "..", "data")
sce_frompython<-zellkonverter::readH5AD(paste0(data_path, 'processed.h5ad'))

We can also do the same into a Seurat object:

In [ ]:
adata_to_seurat<-function(adata){
    raw<-adata$raw$to_adata()
    raw.counts<-t(as.matrix(raw$X))
    rownames(raw.counts)<-rownames(raw$var)
    colnames(raw.counts)<-rownames(raw$obs)

    so<-CreateSeuratObject(counts=raw.counts, assay = 'RNA', meta.data = adata$obs)

    norm.counts<-t(as.matrix(adata$X))

    # whether HVGs were filtered for
    if (length(rownames(so)) == dim(norm.counts)[[1]]){
        rownames(norm.counts)<-rownames(so)
    }else{
        rownames(norm.counts)<-rownames(adata$var[adata$var$highly_variable,])
    }
    colnames(norm.counts)<-colnames(so)

    so@assays$RNA@data<-as(norm.counts, "dgCMatrix")
    so@assays$RNA@meta.features<-adata$var
    
    return(so)
}

In [ ]:
adata <- anndata$read_h5ad(paste0(data_path, 'processed.h5ad'))
covid_data_fromPython <- adata_to_seurat(adata)